<a href="https://colab.research.google.com/github/ritwiks9635/Sequence_Model_Project/blob/main/Text_Generation_with_HuggingFace_GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Experimenting with HuggingFace - Text Generation***

Author: Tucker Arrants

I have recently decided to explore the ins and outs of the 😊 Transformers library and this is the next chapter in that journey. In this notebook, I will explore text generation using a GPT-2 model, which was trained to predict next words on 40GB of Internet text data. The fully trained model is actually not available as the creators were concerned about 'malicious applications of the technology', but there is a much smaller version that is available for enthusiants to play with, which we will use here
In this notebook, we will explore different decoding methods like Beam search, Top-K sampling, and Top-P sampling, demonstrating their performance along the way. This project is a work in progress and I will continually update it as I learn more about text generation. Feel free to comment with any questions/suggestions:
Update: they just released the GPT-3 model (more here) and it has 175 billion parameters and it is shockingly intelligent

In [1]:
! pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 80.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.7 MB/s eta 0:00:00


***Intro***

A language model is a machine learning model that can look at part of a sentence and predict the next word/sequence of words. Much like the autofill features on your iPhone/Android, GPT-2 is capable of next word prediction on a much larger and more sophisticated scale. For reference, the smallest available GPT-2 has 117 million parameters, whereas the largest one (invisible to the public) has over 1.5 billion parameters. The largest one available for public use is half the size of their main GPT-2 model
😊 Transformers makes it very easy to import this model with both PyTorch and TensorFlow - in this notebook we will be using TensorFlow but it is just as easy in PyTorch. Both the model and its Tokenizer can be imported from the transformers library that anyone can get by typing !pip install transformers. Let's see just how simple it is to generate text with a neural network. We begin with our input sequence:

In [2]:
seed = 34

max_len = 70

In [3]:
input_sequence = "I don't know about you, but there's only one thing I want to do after a long day of work"

In [5]:
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer

In [10]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-large")
GPT2 = TFGPT2LMHeadModel.from_pretrained("gpt2-large", pad_token_id=tokenizer.eos_token_id)

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [11]:
GPT2.summary()

Model: "tfgpt2lm_head_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer (TFGPT2MainLay  multiple                  774030080 
 er)                                                             
                                                                 
Total params: 774030080 (2.88 GB)
Trainable params: 774030080 (2.88 GB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


***First Pass (Greedy Search)***

With Greedy search, the word with the highest probability is predicted as the next word i.e. the next word is updated via:

[Math Processing Error]��=��������(�|�1:�−1)
at each timestep [Math Processing Error]�. Let's see how this naive approach performs:

In [12]:
import tensorflow as tf
tf.random.set_seed(seed)

In [13]:
input_idx = tokenizer.encode(input_sequence, return_tensors = "tf")

In [14]:
greedy_output = GPT2.generate(input_idx, max_length = max_len)

In [15]:
print(tokenizer.decode(greedy_output[0], skip_special_tokens = True))

I don't know about you, but there's only one thing I want to do after a long day of work: go to the gym.

I'm not talking about the gym that's right next to my house. I'm talking about the gym that's right next to my office.

I'm not talking about the gym that


And there we go: generating text is that easy. Our results are not great - as we can see, our model starts repeating itself rather quickly. The main issue with Greedy Search is that words with high probabilities can be masked by words in front of them with low probabilities, so the model is unable to explore more diverse combinations of words. We can prevent this by implementing Beam Search:

***Beam Search with N-Gram Penalities***

Beam search is essentially Greedy Search but the model tracks and keeps num_beams of hypotheses at each time step, so the model is able to compare alternative paths as it generates text. We can also include a n-gram penalty by setting no_repeat_ngram_size = 2 which ensures that no 2-grams appear twice. We will also set num_return_sequences = 5 so we can see what the other 5 beams looked like
To use Beam Search, we need only modify some parameters in the generate function:

In [20]:
beam_output = GPT2.generate(
    input_idx,
    max_length = max_len,
    num_beams = 5,
    no_repeat_ngram_size = 2,
    num_return_sequences = 5,
    early_stopping = True)

In [22]:
for i, beam_oupu in enumerate(beam_output):
      print("{}: {}".format(i, tokenizer.decode(beam_oupu, skip_special_tokens=True)))

0: I don't know about you, but there's only one thing I want to do after a long day of work, and that's to sit down and watch a movie."

"I know, I know," you say. "But you're not going to like this one. It's not a good movie. I mean, it's
1: I don't know about you, but there's only one thing I want to do after a long day of work, and that's to sit down and watch a movie."

"I know, I know," you say. "But you're not going to like this one. It's about a guy who has a crush on a girl
2: I don't know about you, but there's only one thing I want to do after a long day of work, and that's to sit down and watch a movie."

"I know, I know," you say. "But you're not going to like this one. It's about a guy who has a crush on a woman
3: I don't know about you, but there's only one thing I want to do after a long day of work, and that's to sit down and watch a movie."

"I know, I know," you say. "But you're not going to like this one. It's about a guy who has a crush on a beautiful
4: I don't kn


Now that's much better! The 5 different beam hypotheses are pretty much all the same, but if we increaed num_beams, then we would see some more variation in the separate beams. But of course, Beam Search is not perfect either. It works well when the legnth of the generated text is more or less constant, like problems in translation or summarization, but not so much for open-ended problems like dialog or story generation (because it is much harder to find a balance between num_beams and no_repeat_ngram_size)
Furthermore, research shows that human languages do not follow this 'high probability word next' distribution. This makes sense - if my words were exactly what you expected them to be, I would be quite a boring person and most people don't want to be boring! The below graph plots the difference of Beam Search and actual human speech:

***Basic Sampling***

Now we will explore indeterministic decodings - sampling. Instead of following a strict path to find the end text with the highest probability, we instead randomly pick the next word by its conditional probability distribution:
[Math Processing Error]��∼�(�|�1:�−1)
However, when we include this randomness, the generated text tends to be incoherent (see more here) so we can include the temperature parameter which increases the chances of high probability words and decreases the chances of low probability words in the sampling:
We just need to set do_sample = True to implement sampling and for demonstration purposes (you'll shortly see why) we set top_k = 0:

In [25]:
# use temperature to decrease the sensitivity to low probability candidates
sample_output = GPT2.generate(
                             input_idx,
                             do_sample = True,
                             max_length = max_len,
                             top_k = 0,
                             temperature = 0.8
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens = True))

Output:
----------------------------------------------------------------------------------------------------
I don't know about you, but there's only one thing I want to do after a long day of work."

"Hmm. Must be quite the choice of words."

"Well, it's not a choice of words, but a need. I can't find the right answer until I find my answer."

"


***Top-K Sampling***

In Top-K sampling, the top k most likely next words are selected and the entire probability mass is shifted to these k words. So instead of increasing the chances of high probability words occuring and decreasing the chances of low probabillity words, we just remove low probability words all together
We just need to set top_k to however many of the top words we want to consider for our conditional probability distribution:

In [26]:
#sample from only top_k most likely words
sample_output = GPT2.generate(
                             input_idx,
                             do_sample = True,
                             max_length = max_len,
                             top_k = 50)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens = True), '...')

Output:
----------------------------------------------------------------------------------------------------
I don't know about you, but there's only one thing I want to do after a long day of work. I want to get out of here and go jogging. To go jogging."

"That may be true, but I don't really have much money to spare!"

"That's true too. Why don ...


***Top-P Sampling***

Top-P sampling (also known as nucleus sampling) is similar to Top-K, but instead of choosing the top k most likely wordsm we choose the smallest set of words whose total probability is larger than p, and then the entire probability mass is shifted to the words in this set
The main difference here is that with Top-K sampling, the size of the set of words is static (obviously) whereas in Top-P sampling, the size of the set can change. To use this sampling method, we just set top_k = 0 and choose a value top_p:

In [27]:
#sample only from 80% most likely words
sample_output = GPT2.generate(
                             input_idx,
                             do_sample = True,
                             max_length = max_len,
                             top_p = 0.8,
                             top_k = 0
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens = True), '...')

Output:
----------------------------------------------------------------------------------------------------
I don't know about you, but there's only one thing I want to do after a long day of work: try out some dessert! Today I've got a total of four different fruit ice creams from The Baker's Dozen. I'm going to share three of them with you, each with a twist.

One was made ...


***Top-K and Top-P Sampling***

As you could have probably guessed, we can use both Top-K and Top-P sampling here. This reduces the chances of us getting weird words (low probability words) while allowing for a dynamic selection size. We need only top a value for both top_k and top_p. We can even include the inital temperature parameter if we want to, Let's now see how our model performs now after adding everything together. We will check the top 5 return to see how diverse our answers are:

In [28]:
#combine both sampling techniques
sample_outputs = GPT2.generate(
                              input_idx,
                              do_sample = True,
                              max_length = 2*max_len,                              #to test how long we can generate and it be coherent
                              #temperature = .7,
                              top_k = 50,
                              top_p = 0.85,
                              num_return_sequences = 5
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}...".format(i, tokenizer.decode(sample_output, skip_special_tokens = True)))
    print('')

Output:
----------------------------------------------------------------------------------------------------
0: I don't know about you, but there's only one thing I want to do after a long day of work and this is one of it. I have to do something else. It's been quite an exciting couple of weeks at the office, haven't I?

Makes you wonder about the people who didn't get the memo that a long day of work is about to turn into a long day of fun....

1: I don't know about you, but there's only one thing I want to do after a long day of work: watch some movies on my bed!

So, I took a trip to my local mall to check out a new line of "couples" furniture. It's the same type of furniture that I saw on an episode of The Bachelor. It's the kind of furniture that makes me think that if I were to be on a reality TV show, I would be dating one of the characters from that show.

The first thing I noticed about the furniture was that there are no chairs. There is only one bed, a desk, a table, and tw

***Benchmark Prompts***

Here, we will see how well the GPT-2 model does when given some more interesting inputs. The following prompts are taken from OpenAI's GPT2 website where they feed them to their full sized GPT2 model (and the results are astounding, I highly recommend you check out their page

In [29]:
max_len = 150

In [30]:
prompt1 = 'In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.'

input_ids = tokenizer.encode(prompt1, return_tensors='tf')

In [31]:
sample_outputs = GPT2.generate(
                              input_ids,
                              do_sample = True,
                              max_length = max_len,                              #to test how long we can generate and it be coherent
                              #temperature = .8,
                              top_k = 50,
                              top_p = 0.85
                              #num_return_sequences = 5
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}...".format(i, tokenizer.decode(sample_output, skip_special_tokens = True)))
    print('')

Output:
----------------------------------------------------------------------------------------------------
0: In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.

According to National Geographic, scientists from the University of São Paulo, Brazil, discovered that the unicorns lived in a valley in the remote Andes mountains, near the village of Mato Grosso do Sul, on the Atlantic coast. The researchers found a number of large horned and bearded animals. They also found traces of humans living nearby, and the animals also carried traces of blood from humans.

The team found the unicorn herd and its human visitors in a valley where they had been watching a herd...



In [32]:
prompt2 = 'Miley Cyrus was caught shoplifting from Abercrombie and Fitch on Hollywood Boulevard today.'

input_ids = tokenizer.encode(prompt2, return_tensors='tf')

In [34]:
sample_outputs = GPT2.generate(
                              input_ids,
                              do_sample = True,
                              max_length = max_len,                              #to test how long we can generate and it be coherent
                              #temperature = .8,
                              top_k = 50,
                              top_p = 0.85
                              #num_return_sequences = 5
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}...".format(i, tokenizer.decode(sample_output, skip_special_tokens = True)))
    print('')

Output:
----------------------------------------------------------------------------------------------------
0: Miley Cyrus was caught shoplifting from Abercrombie and Fitch on Hollywood Boulevard today.

The former Disney star, 22, was spotted leaving the store on her bicycle with $500 in cash.

The news came on a day that Cyrus had been spotted in Beverly Hills with rapper Drake.

The two had met earlier in the day for a photo shoot, which resulted in them meeting the public.

Scroll down for video

The former Disney star was caught shoplifting from Abercrombie & Fitch on Hollywood Boulevard today

Cyrus was spotted leaving the store on her bicycle with $500 in cash

Cyrus was seen wearing a yellow top and white shorts with a blue and white striped...



In [35]:
prompt3 = 'Legolas and Gimli advanced on the orcs, raising their weapons with a harrowing war cry.'

input_ids = tokenizer.encode(prompt3, return_tensors='tf')

In [36]:
sample_outputs = GPT2.generate(
                              input_ids,
                              do_sample = True,
                              max_length = max_len,                              #to test how long we can generate and it be coherent
                              #temperature = .8,
                              top_k = 50,
                              top_p = 0.85
                              #num_return_sequences = 5
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}...".format(i, tokenizer.decode(sample_output, skip_special_tokens = True)))
    print('')

Output:
----------------------------------------------------------------------------------------------------
0: Legolas and Gimli advanced on the orcs, raising their weapons with a harrowing war cry.

All of the orc warbands, including the ones from which they were drawn, began charging, leaving the Alliance's ranks behind. The two sides battled with the orc warriors in close combat, and even though the two sides continued to clash, the battle seemed to go on forever. Then, all at once, the entire horde of orcs turned, as if to charge the approaching orcs. The battle began again, and the battle continued. The battle raged on for a good ten minutes or so, until the orcs were surrounded and routed. The battle continued on for another ten minutes or so, until all of the orcs were dead. All of the...



In [37]:
prompt4 = "For today’s homework assignment, please describe the reasons for the US Civil War."

input_ids = tokenizer.encode(prompt4, return_tensors='tf')

In [38]:
sample_outputs = GPT2.generate(
                              input_ids,
                              do_sample = True,
                              max_length = max_len,                              #to test how long we can generate and it be coherent
                              #temperature = .8,
                              top_k = 50,
                              top_p = 0.85
                              #num_return_sequences = 5
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}...".format(i, tokenizer.decode(sample_output, skip_special_tokens = True)))
    print('')

Output:
----------------------------------------------------------------------------------------------------
0: For today’s homework assignment, please describe the reasons for the US Civil War.

For more from The Week's Power Lunch, click here.

Follow @dgbxny...




Now we can see why OpenAI is hesitate to release their full scale model (recall, over 1.5 billion parameters) to the general public...it has the potential to do a lot of harm when used with malevolent intentions. Luckily, they have still provided us smaller versions that allow us to demonstrate just how powerful Transformer models are when applied to natural language and how they provide an analytical space from which to study linguistics.
Feel free to fork this notebook and experiment with your own inputs and model parameters to see what type of computer generated stories you can create!

***References***

Below are the only references you need to implement your own GPT-2 model for text generation

https://arxiv.org/abs/1904.09751
https://openai.com/blog/better-language-models/
https://huggingface.co/transformers/model_doc/gpt2.html
https://huggingface.co/blog/how-to-generate